<a href="https://colab.research.google.com/github/Debottam/MachinLearningEx/blob/master/GANEx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# example of loading the mnist dataset
from keras.datasets.mnist import load_data
# load the images into memory
(trainX, trainy), (testX, testy) = load_data()
# summarize the shape of the dataset
print('Train', trainX.shape, trainy.shape)
print('Test', testX.shape, testy.shape)

Using TensorFlow backend.


11501568/11490434 [==============================] - 0s 0us/step
('Train', (60000, 28, 28), (60000,))
('Test', (10000, 28, 28), (10000,))
